In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import pandas_ta as ta
import joblib
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Inicializar MetaTrader 5
if not mt5.initialize():
    print("Failed to initialize MT5, error code =", mt5.last_error())
    quit()

# Definir o símbolo e o intervalo de tempo (ex: 1 minuto)
symbol = "WDOX24"
timeframe = mt5.TIMEFRAME_M1  # 1 minuto
num_bars = 500  # Número de candles para carregar

In [ ]:
# Função para enviar ordens de compra
def buy_order():
    lot = 1.0  # Tamanho do lote
    price = mt5.symbol_info_tick(symbol).ask  # Preço de compra
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": price - 100,  # Stop Loss (ajustar de acordo com sua estratégia)
        "tp": price + 100,  # Take Profit (ajustar de acordo com sua estratégia)
        "deviation": 10,
        "magic": 123456,
        "comment": "Buy order by ML model",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print("Compra realizada com sucesso!")
    else:
        print(f"Erro ao executar compra: {result.retcode}")

# Função para enviar ordens de venda
def sell_order():
    lot = 1.0  # Tamanho do lote
    price = mt5.symbol_info_tick(symbol).bid  # Preço de venda
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": price + 100,  # Stop Loss (ajustar de acordo com sua estratégia)
        "tp": price - 100,  # Take Profit (ajustar de acordo com sua estratégia)
        "deviation": 10,
        "magic": 123456,
        "comment": "Sell order by ML model",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print("Venda realizada com sucesso!")
    else:
        print(f"Erro ao executar venda: {result.retcode}")


In [ ]:
import time
# Obter dados ao vivo de 1 minuto
while True:
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    if rates is None:
        print(f"Error: {mt5.last_error()}")
        mt5.shutdown()
        exit()
    
    # Convertendo os dados para um DataFrame para facilitar
    data = pd.DataFrame(rates)
    
    # Adicionando indicadores técnicos ao DataFrame
    data['RSI'] = ta.rsi(data['close'], length=5)
    data['EMAF'] = ta.ema(data['close'], length=10)
    data['EMAM'] = ta.ema(data['close'], length=40)
    data['EMAS'] = ta.ema(data['close'], length=80)
    
    # Criando o 'Target' (diferença entre fechamento e abertura)
    data['Target'] = data['close'] - data['open']
    
    # Deslocando a coluna 'Target' para o próximo candle
    data['Target'] = data['Target'].shift(-1)
    
    # Criando a 'TargetClass' (1 para alta, 0 para queda)
    data['TargetClass'] = [1 if data['Target'][i] > 0 else 0 for i in range(len(data))]
    
    # Coluna 'TargetNextClose' que contém o valor de fechamento do próximo candle
    data['TargetNextClose'] = data['close'].shift(-1)
    
    # Removendo valores nulos
    data.dropna(inplace=True)
    
    # Resetando o índice após a remoção de linhas
    data.reset_index(drop=True, inplace=True)
    
    # Removendo colunas desnecessárias
    data.drop(['time', 'spread', 'tick_volume', 'real_volume'], axis=1, inplace=True)
    
    # Normalizar apenas as colunas de entrada (open, high, low, close, RSI, EMAF, EMAM, EMAS)
    sc = MinMaxScaler(feature_range=(0, 1))
    data_scaled = sc.fit_transform(data[['open', 'high', 'low', 'close', 'RSI', 'EMAF', 'EMAM', 'EMAS']])
    
    # Criar as features (X) para os novos dados ao vivo
    backcandles = 5  # Mesmo número de candles usado no treino
    X_live = []
    for j in range(data_scaled.shape[1]):  # Para todas as colunas normalizadas (open até EMAS)
        X_live.append([])
        for i in range(backcandles, data_scaled.shape[0]):
            X_live[j].append(data_scaled[i-backcandles:i, j])
    
    X_live = np.moveaxis(X_live, [0], [2])
    X_live = np.array(X_live)
    
    # Carregar o modelo treinado
    model, ref_cols, target = joblib.load("modelWDO2024_past01.pkl")
    
    # Fazer previsões com os dados ao vivo
    predictions = model.predict(X_live)
    
    # Desnormalizar as previsões (considerando que o target é o preço do fechamento)
    sc_target = MinMaxScaler(feature_range=(0, 1))
    sc_target.fit(data[['TargetNextClose']])  # Usar o escalador treinado nos valores de fechamento
    predictions_descaled = sc_target.inverse_transform(predictions)

        # Prever a próxima ação do mercado
    predictions = model.predict(X_live)  # Use o modelo que você treinou
    next_prediction = predictions[-1]

    # Decidir se compra ou vende
    if next_prediction > 0.5:  # Se o modelo prever alta, executa compra
        print("Modelo prevê alta, comprando...")
        buy_order()
    elif next_prediction <= 0.5:  # Se o modelo prever queda, executa venda
        print("Modelo prevê queda, vendendo...")
        sell_order()

    # Aguardar um tempo antes de fazer a próxima previsão (por exemplo, a cada minuto)
    time.sleep(60)

In [38]:
# Fechar a conexão com o MetaTrader 5
mt5.shutdown()


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Previsão desnormalizada do modelo (próximo fechamento): 133815.203125


True

In [2]:
import MetaTrader5 as mt5
import pandas as pd
import pandas_ta as ta
import joblib
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time

# Inicializar MetaTrader 5
if not mt5.initialize():
    print("Failed to initialize MT5, error code =", mt5.last_error())
    quit()

# Definir o símbolo e o intervalo de tempo
symbol = "WDOX24"
timeframe = mt5.TIMEFRAME_M1  # 1 minuto
num_bars = 500  # Número de candles para carregar

def buy_order():
    lot = 1.0
    price = mt5.symbol_info_tick(symbol).ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": price - 100,
        "tp": price + 100,
        "deviation": 10,
        "magic": 123456,
        "comment": "Buy order by ML model",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Compra realizada com sucesso! Preço: {price}")
    else:
        print(f"Erro ao executar compra: {result.retcode}")

def sell_order():
    lot = 1.0
    price = mt5.symbol_info_tick(symbol).bid
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": price + 100,
        "tp": price - 100,
        "deviation": 10,
        "magic": 123456,
        "comment": "Sell order by ML model",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Venda realizada com sucesso! Preço: {price}")
    else:
        print(f"Erro ao executar venda: {result.retcode}")

# Variável para armazenar a última operação
last_operation = None

# Loop principal
while True:
    try:
        # Obter dados ao vivo
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        if rates is None:
            print(f"Error: {mt5.last_error()}")
            mt5.shutdown()
            exit()
        
        # Convertendo os dados para DataFrame
        data = pd.DataFrame(rates)
        
        # Adicionando indicadores técnicos
        data['RSI'] = ta.rsi(data['close'], length=5)
        data['EMAF'] = ta.ema(data['close'], length=10)
        data['EMAM'] = ta.ema(data['close'], length=40)
        data['EMAS'] = ta.ema(data['close'], length=80)
        
        # Criando o Target (diferença entre fechamento e abertura)
        data['Target'] = data['close'] - data['open']
        data['Target'] = data['Target'].shift(-1)
        
        # Criando a TargetClass (1 para alta, 0 para queda)
        data['TargetClass'] = [1 if x > 0 else 0 for x in data['Target']]

        # Coluna 'TargetNextClose' que contém o valor de fechamento do próximo candle
        data['TargetNextClose'] = data['close'].shift(-1)
        
        # Limpeza dos dados
        data.dropna(inplace=True)
        data.reset_index(drop=True, inplace=True)
        data.drop(['time', 'spread', 'tick_volume', 'real_volume'], axis=1, inplace=True)
        
        # Normalização
        sc = MinMaxScaler(feature_range=(0, 1))
        data_scaled = sc.fit_transform(data[['open', 'high', 'low', 'close', 'RSI', 'EMAF', 'EMAM', 'EMAS']])
        
        # Preparar dados para previsão
        backcandles = 5
        X_live = []
        for j in range(data_scaled.shape[1]):
            X_live.append([])
            for i in range(backcandles, data_scaled.shape[0]):
                X_live[j].append(data_scaled[i-backcandles:i, j])
        
        X_live = np.moveaxis(X_live, [0], [2])
        X_live = np.array(X_live)
        
        # Carregar modelo e fazer previsão
        model, ref_cols, target = joblib.load("modelWDO2024_past01.pkl")
        predictions = model.predict(X_live)
        
        # Obter última previsão e dados relevantes
        current_prediction = float(predictions[-1])
        current_price = mt5.symbol_info_tick(symbol).last
        current_target = float(data['TargetNextClose'].iloc[-1])
        
        print(f"Previsão atual: {current_prediction:.4f}, Target Next Close atual: {current_target:.4f}, Preço atual: {current_price}")
        
        # Lógica de trading baseada no Target
        if current_target > 0:  # Target positivo indica tendência de alta
            if last_operation != "BUY":
                print(f"Sinal de COMPRA detectado - Target: {current_target:.4f}")
                buy_order()
                last_operation = "BUY"
        elif current_target < 0:  # Target negativo indica tendência de baixa
            if last_operation != "SELL":
                print(f"Sinal de VENDA detectado - Target: {current_target:.4f}")
                sell_order()
                last_operation = "SELL"
        else:
            print(f"Aguardando sinal mais claro do mercado... Target atual: {current_target:.4f}")
        
        # Aguardar antes da próxima iteração
        time.sleep(60)
        
    except Exception as e:
        print(f"Erro durante a execução: {str(e)}")
        time.sleep(60)
        continue

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Previsão atual: 0.3766, Target Next Close atual: 5666.5000, Preço atual: 5666.5
Sinal de COMPRA detectado - Target: 5666.5000
Compra realizada com sucesso! Preço: 5666.5


C:\Users\Bhakt\AppData\Local\Temp\ipykernel_7752\2431307759.py:122: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_prediction = float(predictions[-1])


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Previsão atual: 0.3704, Target Next Close atual: 5668.0000, Preço atual: 5668.0


C:\Users\Bhakt\AppData\Local\Temp\ipykernel_7752\2431307759.py:122: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_prediction = float(predictions[-1])


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Previsão atual: 0.3723, Target Next Close atual: 5667.0000, Preço atual: 5667.0


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Previsão atual: 0.6263, Target Next Close atual: 5674.0000, Preço atual: 5674.5


C:\Users\Bhakt\AppData\Local\Temp\ipykernel_7752\2431307759.py:122: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_prediction = float(predictions[-1])


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Previsão atual: 0.6203, Target Next Close atual: 5671.5000, Preço atual: 5671.0


C:\Users\Bhakt\AppData\Local\Temp\ipykernel_7752\2431307759.py:122: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_prediction = float(predictions[-1])


KeyboardInterrupt: 